In [1]:
!pip install torch torchaudio torchvision sentence-transformers
!pip install sentence-transformers faiss-gpu  # or faiss-cpu if GPU is not available

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 779.1/779.1 MB 1.1 MB/s eta 0:00:0000:0100:03
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 7.9 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 9.0 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 8.1 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 5.7 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.0/168.0 MB 2.5 MB/s eta 0:00:0000:0100:02
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 2.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 1.5 MB/s eta 0:00:0000:0100:02
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 7.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━

  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.3.0
    Uninstalling typing_extensions-4.3.0:
      Successfully uninstalled typing_extensions-4.3.0
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2021.7.0
    Uninstalling fsspec-2021.7.0:
      Successfully uninstalled fsspec-2021.7.0
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.0.12
    Uninstalling huggingface-hub-0.0.12:
      Successfully uninstalled huggingface-hub-0.0.12
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.10.2
    Uninstalling tokenizers-0.10.2:
      Successfully uninstalled tokenizers-0.10.2
  Attempting uninstall: transformers
    Found existing installation: transformers 4.9.1
    Uninstalling transformers-4.9.1:
      Successfully uninstalled transformers-4.9.1
--- Logging error ---
Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/p

--- Logging error ---
Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/pip/_internal/utils/logging.py", line 177, in emit
    self.console.print(renderable, overflow="ignore", crop=False, style=style)
  File "/usr/local/lib/python3.8/dist-packages/pip/_vendor/rich/console.py", line 1673, in print
    extend(render(renderable, render_options))
  File "/usr/local/lib/python3.8/dist-packages/pip/_vendor/rich/console.py", line 1305, in render
    for render_output in iter_render:
  File "/usr/local/lib/python3.8/dist-packages/pip/_internal/utils/logging.py", line 134, in __rich_console__
    for line in lines:
  File "/usr/local/lib/python3.8/dist-packages/pip/_vendor/rich/segment.py", line 249, in split_lines
    for segment in segments:
  File "/usr/local/lib/python3.8/dist-packages/pip/_vendor/rich/console.py", line 1283, in render
    renderable = rich_cast(renderable)
  File "/usr/local/lib/python3.8/dist-packages/pip/_vendor/rich/protocol.py", line 36

In [ ]:
# Necessary imports
import torch
import pandas as pd
import numpy as np
import time
from torch.utils.data import DataLoader, Dataset
from torch import nn, optim
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

# Initialize the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Custom Dataset class for handling triplet data
class TripletDataset(Dataset):
    def __init__(self, dataframe):
        self.dataframe = dataframe

    def __getitem__(self, idx):
        data = self.dataframe.iloc[idx]
        return {
            'anchor': data['anchor'],
            'positive': data['positive'],
            'negative': data['negative']
        }

    def __len__(self):
        return len(self.dataframe)

# Load the Sentence Transformer model
model = SentenceTransformer('all-MiniLM-L6-v2').to(device)

# Triplet Loss Model definition
class TripletLossModel(nn.Module):
    def __init__(self):
        super(TripletLossModel, self).__init__()
        self.embedding_model = model

    def forward(self, anchor, positive, negative):
        anchor_emb = self.embedding_model.encode(anchor, convert_to_tensor=True, device=device)
        positive_emb = self.embedding_model.encode(positive, convert_to_tensor=True, device=device)
        negative_emb = self.embedding_model.encode(negative, convert_to_tensor=True, device=device)
        return anchor_emb, positive_emb, negative_emb

# Load dataset and prepare DataLoader
df_triplets = pd.read_csv("triplet_data.csv")
triplet_dataset = TripletDataset(df_triplets)
triplet_loader = DataLoader(triplet_dataset, batch_size=10, shuffle=True, num_workers=5)

# Model, Loss, and Optimizer setup
triplet_model = TripletLossModel().to(device)
loss_function = nn.TripletMarginLoss(margin=1.0)
optimizer = optim.Adam(triplet_model.parameters(), lr=0.01)




In [ ]:
# Function to preprocess and embed texts using Sentence Transformers
def preprocess_and_embed(texts):
    embeddings = model.encode(texts, convert_to_tensor=False, show_progress_bar=True, device=device)
    return embeddings

# Load data for evaluation
df = pd.read_csv("cleaned_dataset_with_renamed_description.csv").dropna(subset=['title', 'description'])
combined_texts = df['title'] + " " + df['description']
book_embeddings = preprocess_and_embed(combined_texts.tolist())

# Load test data
test_df = pd.read_csv("enhanced_test_data_n=4.csv")
# Function to evaluate the recommendation system
def evaluate_recommendations(test_df, book_embeddings, df, top_n=50):
    # Convert the 'Recommended_Books' column from strings to sets of titles
    test_df['Recommended_Books'] = test_df['Recommended_Books'].apply(lambda x: set(x.split(', ')))

    # Initialize list to store accuracy
    accuracy_list = []

    # Loop through each row in the test dataframe
    for _, row in test_df.iterrows():
        # Get the combined description and true recommended books
        user_input = row['Combined_Description']
        true_books = row['Recommended_Books']

        # Get the user's embedding for the given combined description
        user_embedding = preprocess_and_embed([user_input])[0]  # Access the first (and only) embedding

        # Calculate cosine similarity between the user's embedding and book embeddings
        similarities = cosine_similarity([user_embedding], book_embeddings).flatten()

        # Get indices of the top_n most similar books
        top_indices = np.argsort(similarities)[-top_n:][::-1]

        # Get the top recommended books' titles
        predicted_books = set(df.iloc[top_indices]['title'])

        # Calculate accuracy by checking if true recommended books are present in the top_n predicted books
        correct_recommendations = true_books.intersection(predicted_books)
        accuracy = len(correct_recommendations) / len(true_books)

        # Append accuracy to the list
        accuracy_list.append(accuracy)

    # Calculate the average accuracy
    average_accuracy = np.mean(accuracy_list)

    # Print and return the average accuracy
    print(f"Average Accuracy: {average_accuracy:.4f}")
    return {
        "Average Accuracy": average_accuracy,
    }

print("for n=4")
# Evaluate the recommendation system using the test data
results = evaluate_recommendations(test_df, book_embeddings, df, top_n=50)

In [ ]:
# Function to preprocess and embed texts using Sentence Transformers
def preprocess_and_embed(texts):
    embeddings = model.encode(texts, convert_to_tensor=False, show_progress_bar=True, device=device)
    return embeddings

# Load data for evaluation
df = pd.read_csv("cleaned_dataset_with_renamed_description.csv").dropna(subset=['title', 'description'])
combined_texts = df['title'] + " " + df['description']
book_embeddings = preprocess_and_embed(combined_texts.tolist())

# Load test data
test_df = pd.read_csv("enhanced_test_data_n=4.csv")
# Function to evaluate the recommendation system
def evaluate_recommendations(test_df, book_embeddings, df, top_n=40):
    # Convert the 'Recommended_Books' column from strings to sets of titles
    test_df['Recommended_Books'] = test_df['Recommended_Books'].apply(lambda x: set(x.split(', ')))

    # Initialize list to store accuracy
    accuracy_list = []

    # Loop through each row in the test dataframe
    for _, row in test_df.iterrows():
        # Get the combined description and true recommended books
        user_input = row['Combined_Description']
        true_books = row['Recommended_Books']

        # Get the user's embedding for the given combined description
        user_embedding = preprocess_and_embed([user_input])[0]  # Access the first (and only) embedding

        # Calculate cosine similarity between the user's embedding and book embeddings
        similarities = cosine_similarity([user_embedding], book_embeddings).flatten()

        # Get indices of the top_n most similar books
        top_indices = np.argsort(similarities)[-top_n:][::-1]

        # Get the top recommended books' titles
        predicted_books = set(df.iloc[top_indices]['title'])

        # Calculate accuracy by checking if true recommended books are present in the top_n predicted books
        correct_recommendations = true_books.intersection(predicted_books)
        accuracy = len(correct_recommendations) / len(true_books)

        # Append accuracy to the list
        accuracy_list.append(accuracy)

    # Calculate the average accuracy
    average_accuracy = np.mean(accuracy_list)

    # Print and return the average accuracy
    print(f"Average Accuracy: {average_accuracy:.4f}")
    return {
        "Average Accuracy": average_accuracy,
    }

print("for n=4")
# Evaluate the recommendation system using the test data
results = evaluate_recommendations(test_df, book_embeddings, df, top_n=40)

In [ ]:
# Function to preprocess and embed texts using Sentence Transformers
def preprocess_and_embed(texts):
    embeddings = model.encode(texts, convert_to_tensor=False, show_progress_bar=True, device=device)
    return embeddings

# Load data for evaluation
df = pd.read_csv("cleaned_dataset_with_renamed_description.csv").dropna(subset=['title', 'description'])
combined_texts = df['title'] + " " + df['description']
book_embeddings = preprocess_and_embed(combined_texts.tolist())

# Load test data
test_df = pd.read_csv("enhanced_test_data_n=4.csv")
# Function to evaluate the recommendation system
def evaluate_recommendations(test_df, book_embeddings, df, top_n=60):
    # Convert the 'Recommended_Books' column from strings to sets of titles
    test_df['Recommended_Books'] = test_df['Recommended_Books'].apply(lambda x: set(x.split(', ')))

    # Initialize list to store accuracy
    accuracy_list = []

    # Loop through each row in the test dataframe
    for _, row in test_df.iterrows():
        # Get the combined description and true recommended books
        user_input = row['Combined_Description']
        true_books = row['Recommended_Books']

        # Get the user's embedding for the given combined description
        user_embedding = preprocess_and_embed([user_input])[0]  # Access the first (and only) embedding

        # Calculate cosine similarity between the user's embedding and book embeddings
        similarities = cosine_similarity([user_embedding], book_embeddings).flatten()

        # Get indices of the top_n most similar books
        top_indices = np.argsort(similarities)[-top_n:][::-1]

        # Get the top recommended books' titles
        predicted_books = set(df.iloc[top_indices]['title'])

        # Calculate accuracy by checking if true recommended books are present in the top_n predicted books
        correct_recommendations = true_books.intersection(predicted_books)
        accuracy = len(correct_recommendations) / len(true_books)

        # Append accuracy to the list
        accuracy_list.append(accuracy)

    # Calculate the average accuracy
    average_accuracy = np.mean(accuracy_list)

    # Print and return the average accuracy
    print(f"Average Accuracy: {average_accuracy:.4f}")
    return {
        "Average Accuracy": average_accuracy,
    }

print("for n=4")
# Evaluate the recommendation system using the test data
results = evaluate_recommendations(test_df, book_embeddings, df, top_n=60)

In [ ]:
# Function to preprocess and embed texts using Sentence Transformers
def preprocess_and_embed(texts):
    embeddings = model.encode(texts, convert_to_tensor=False, show_progress_bar=True, device=device)
    return embeddings

# Load data for evaluation
df = pd.read_csv("cleaned_dataset_with_renamed_description.csv").dropna(subset=['title', 'description'])
combined_texts = df['title'] + " " + df['description']
book_embeddings = preprocess_and_embed(combined_texts.tolist())

# Load test data
test_df = pd.read_csv("enhanced_test_data_n=4.csv")
# Function to evaluate the recommendation system
def evaluate_recommendations(test_df, book_embeddings, df, top_n=10):
    # Convert the 'Recommended_Books' column from strings to sets of titles
    test_df['Recommended_Books'] = test_df['Recommended_Books'].apply(lambda x: set(x.split(', ')))

    # Initialize list to store accuracy
    accuracy_list = []

    # Loop through each row in the test dataframe
    for _, row in test_df.iterrows():
        # Get the combined description and true recommended books
        user_input = row['Combined_Description']
        true_books = row['Recommended_Books']

        # Get the user's embedding for the given combined description
        user_embedding = preprocess_and_embed([user_input])[0]  # Access the first (and only) embedding

        # Calculate cosine similarity between the user's embedding and book embeddings
        similarities = cosine_similarity([user_embedding], book_embeddings).flatten()

        # Get indices of the top_n most similar books
        top_indices = np.argsort(similarities)[-top_n:][::-1]

        # Get the top recommended books' titles
        predicted_books = set(df.iloc[top_indices]['title'])

        # Calculate accuracy by checking if true recommended books are present in the top_n predicted books
        correct_recommendations = true_books.intersection(predicted_books)
        accuracy = len(correct_recommendations) / len(true_books)

        # Append accuracy to the list
        accuracy_list.append(accuracy)

    # Calculate the average accuracy
    average_accuracy = np.mean(accuracy_list)

    # Print and return the average accuracy
    print(f"Average Accuracy: {average_accuracy:.4f}")
    return {
        "Average Accuracy": average_accuracy,
    }

print("for n=4")
# Evaluate the recommendation system using the test data
results = evaluate_recommendations(test_df, book_embeddings, df, top_n=10)

In [ ]:
# Function to preprocess and embed texts using Sentence Transformers
def preprocess_and_embed(texts):
    embeddings = model.encode(texts, convert_to_tensor=False, show_progress_bar=True, device=device)
    return embeddings

# Load data for evaluation
df = pd.read_csv("cleaned_dataset_with_renamed_description.csv").dropna(subset=['title', 'description'])
combined_texts = df['title'] + " " + df['description']
book_embeddings = preprocess_and_embed(combined_texts.tolist())

# Load test data
test_df = pd.read_csv("enhanced_test_data_n=4.csv")
# Function to evaluate the recommendation system
def evaluate_recommendations(test_df, book_embeddings, df, top_n=20):
    # Convert the 'Recommended_Books' column from strings to sets of titles
    test_df['Recommended_Books'] = test_df['Recommended_Books'].apply(lambda x: set(x.split(', ')))

    # Initialize list to store accuracy
    accuracy_list = []

    # Loop through each row in the test dataframe
    for _, row in test_df.iterrows():
        # Get the combined description and true recommended books
        user_input = row['Combined_Description']
        true_books = row['Recommended_Books']

        # Get the user's embedding for the given combined description
        user_embedding = preprocess_and_embed([user_input])[0]  # Access the first (and only) embedding

        # Calculate cosine similarity between the user's embedding and book embeddings
        similarities = cosine_similarity([user_embedding], book_embeddings).flatten()

        # Get indices of the top_n most similar books
        top_indices = np.argsort(similarities)[-top_n:][::-1]

        # Get the top recommended books' titles
        predicted_books = set(df.iloc[top_indices]['title'])

        # Calculate accuracy by checking if true recommended books are present in the top_n predicted books
        correct_recommendations = true_books.intersection(predicted_books)
        accuracy = len(correct_recommendations) / len(true_books)

        # Append accuracy to the list
        accuracy_list.append(accuracy)

    # Calculate the average accuracy
    average_accuracy = np.mean(accuracy_list)

    # Print and return the average accuracy
    print(f"Average Accuracy: {average_accuracy:.4f}")
    return {
        "Average Accuracy": average_accuracy,
    }

print("for n=4")
# Evaluate the recommendation system using the test data
results = evaluate_recommendations(test_df, book_embeddings, df, top_n=20)

In [ ]:
# Function to preprocess and embed texts using Sentence Transformers
def preprocess_and_embed(texts):
    embeddings = model.encode(texts, convert_to_tensor=False, show_progress_bar=True, device=device)
    return embeddings

# Load data for evaluation
df = pd.read_csv("cleaned_dataset_with_renamed_description.csv").dropna(subset=['title', 'description'])
combined_texts = df['title'] + " " + df['description']
book_embeddings = preprocess_and_embed(combined_texts.tolist())

# Load test data
test_df = pd.read_csv("enhanced_test_data_n=4.csv")
# Function to evaluate the recommendation system
def evaluate_recommendations(test_df, book_embeddings, df, top_n=30):
    # Convert the 'Recommended_Books' column from strings to sets of titles
    test_df['Recommended_Books'] = test_df['Recommended_Books'].apply(lambda x: set(x.split(', ')))

    # Initialize list to store accuracy
    accuracy_list = []

    # Loop through each row in the test dataframe
    for _, row in test_df.iterrows():
        # Get the combined description and true recommended books
        user_input = row['Combined_Description']
        true_books = row['Recommended_Books']

        # Get the user's embedding for the given combined description
        user_embedding = preprocess_and_embed([user_input])[0]  # Access the first (and only) embedding

        # Calculate cosine similarity between the user's embedding and book embeddings
        similarities = cosine_similarity([user_embedding], book_embeddings).flatten()

        # Get indices of the top_n most similar books
        top_indices = np.argsort(similarities)[-top_n:][::-1]

        # Get the top recommended books' titles
        predicted_books = set(df.iloc[top_indices]['title'])

        # Calculate accuracy by checking if true recommended books are present in the top_n predicted books
        correct_recommendations = true_books.intersection(predicted_books)
        accuracy = len(correct_recommendations) / len(true_books)

        # Append accuracy to the list
        accuracy_list.append(accuracy)

    # Calculate the average accuracy
    average_accuracy = np.mean(accuracy_list)

    # Print and return the average accuracy
    print(f"Average Accuracy: {average_accuracy:.4f}")
    return {
        "Average Accuracy": average_accuracy,
    }

print("for n=4")
# Evaluate the recommendation system using the test data
results = evaluate_recommendations(test_df, book_embeddings, df, top_n=30)